Trabalho realizado por: Bárbara Freixo, PG49169

Esta implementação define uma class personalizada para a implementação de uma árvore de decisão chamada CustomDecisionTreeClassifier. Ela apresenta várias opções para personalizar a construção da árvore de decisão, como escolher o impurity criterion (entropy ou Gini index), o splitting method (information gain ou gain ratio) e parâmetros como maximum depth, minimum number of samples for splitting, pre-prunning e post-pruning.

Principais componentes do código:

* Inicializador (init): Define os parâmetros da árvore de decisão, como critério de impureza, método de divisão, profundidade máxima, número mínimo de amostras para divisão, entre outros.
* Funções para calcular entropia, índice de Gini e taxa de ganho.
* Função _best_split: Seleciona o melhor ponto de divisão para um conjunto de dados com base no critério de impureza.
* Função _build_tree: Constrói a árvore de decisão recursivamente com base nos parâmetros definidos.
* Funções para pré-poda (_chi_squared_test) e pós-poda (_reduced_error_pruning e _pessimistic_error_pruning) da árvore.
* Função fit: Treina a árvore de decisão com base no conjunto de treinamento fornecido.
* Função predict: Faz previsões para um conjunto de dados com base na árvore de decisão treinada.

**Código:**

In [1]:
import numpy as np
from scipy.stats import chi2_contingency

class CustomDecisionTreeClassifier:

    def __init__(self, criterion='entropy', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None,
                 max_leaf_nodes=None, class_threshold=0.5, pre_pruning=None, post_pruning=None, min_size=None):
        # Inicialização dos parâmetros da árvore de decisão
        self.criterion = criterion
        self.splitter = splitter
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.max_leaf_nodes = max_leaf_nodes
        self.class_threshold = class_threshold
        self.pre_pruning = pre_pruning
        self.post_pruning = post_pruning
        self.min_size = min_size
        self.tree = None

    def _calculate_entropy(self, y):
        # Calcula a entropia de um conjunto de dados y
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / counts.sum()
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def _calculate_gini(self, y):
         # Calcula o índice de Gini de um conjunto de dados y
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / counts.sum()
        gini = 1 - np.sum(probabilities**2)
        return gini

    def _gain_ratio(self, gain, y, y_left, y_right):
        # Calcula o ganho da razão para um determinado ganho e partições de dados
        split_info = -((len(y_left) / len(y)) * np.log2(len(y_left) / len(y)) + (len(y_right) / len(y)) * np.log2(len(y_right) / len(y)))
        gain_ratio = gain / split_info
        return gain_ratio

    def _best_split(self, X, y):
        # Encontra o melhor ponto de divisão para um conjunto de dados X e y
        best_value = 0
        best_feature_idx = -1
        best_threshold = None
        n_features = X.shape[1]

        if self.criterion == 'entropy':
            impurity = self._calculate_entropy(y)
        elif self.criterion == 'gini':
            impurity = self._calculate_gini(y)
        else:
            raise ValueError(f"Invalid criterion '{self.criterion}', use 'entropy' or 'gini'")

        for feature_idx in range(n_features):
            feature_values = X[:, feature_idx]
            for threshold in np.unique(feature_values):
                mask = feature_values < threshold
                y_left = y[mask]
                y_right = y[~mask]

                if len(y_left) < self.min_samples_split or len(y_right) < self.min_samples_split:
                    continue

                left_impurity = self._calculate_entropy(y_left) if self.criterion == 'entropy' else self._calculate_gini(y_left)
                right_impurity = self._calculate_entropy(y_right) if self.criterion == 'entropy' else self._calculate_gini(y_right)
                weighted_impurity = (len(y_left) * left_impurity + len(y_right) * right_impurity) / len(y)

                gain = impurity - weighted_impurity

                if self.splitter == 'gain_ratio':
                    value = self._gain_ratio(gain, y, y_left, y_right)
                else:
                    value = gain

                if value > best_value:
                    best_value = value
                    best_feature_idx = feature_idx
                    best_threshold = threshold

        return best_feature_idx, best_threshold

    def _build_tree(self, X, y, depth, n_nodes):
        # Constrói a árvore de decisão recursivamente
        if depth == self.max_depth or len(np.unique(y)) == 1 or len(y) < self.min_samples_split or n_nodes == self.max_leaf_nodes or (self.min_size is not None and len(y) < self.min_size):
            majority_class = self._majority_voting_with_threshold(y)
            return {'label': majority_class if majority_class is not None else np.argmax(np.bincount(y))}
        
        feature_idx, threshold = self._best_split(X, y)
        
        if threshold is None:
            return {'label': np.argmax(np.bincount(y))}
        
        if self.pre_pruning == 'independence':
            p_value = self._chi_squared_test(X, y, feature_idx, threshold)
            if p_value > self.class_threshold:
                return {'label': np.argmax(np.bincount(y))}

        mask = X[:, feature_idx] < threshold
        left = self._build_tree(X[mask], y[mask], depth + 1, n_nodes + 1)
        right = self._build_tree(X[~mask], y[~mask], depth + 1, n_nodes + 1)

        return {'feature_idx': feature_idx, 'threshold': threshold, 'left': left, 'right': right}
                                
    def _majority_voting_with_threshold(self, y):
         # Realiza votação majoritária com um limiar para decidir a classe
        class_counts = np.bincount(y)
        max_count = np.max(class_counts)
        majority_class = np.argmax(class_counts)
        return majority_class if max_count / len(y) > self.class_threshold else None

    def _chi_squared_test(self, X, y, feature_idx, threshold):
         # Realiza o teste do qui-quadrado para verificar a independência das variáveis
        contingency_table = np.zeros((2, len(np.unique(y))))
        mask = X[:, feature_idx] < threshold
        for i, class_label in enumerate(np.unique(y)):
            contingency_table[0, i] = np.sum(y[mask] == class_label)
            contingency_table[1, i] = np.sum(y[~mask] == class_label)
        
        chi2, p_value, _, _ = chi2_contingency(contingency_table)
        return p_value

    def fit(self, X, y):
        # Ajusta o modelo aos dados de treinamento
        self.tree = self._build_tree(X, y, depth=0, n_nodes=0)

    def _predict_sample(self, x, node):
         # Prevê a classe de uma única amostra
        if 'label' in node:
            return node['label']

        if x[node['feature_idx']] < node['threshold']:
            return self._predict_sample(x, node['left'])
        else:
            return self._predict_sample(x, node['right'])
    
    def predict(self, X):
        # Prevê a classe para um conjunto de dados X
        predictions = [self._predict_sample(x, self.tree) for x in X]
        return np.array(predictions)

    def _reduced_error_pruning(self, node, X, y, parent_majority_label=None):
      # Realiza poda por redução de erro
      if y.size == 0 or 'label' in node:
          return node

      feature_idx = node['feature_idx']
      threshold = node['threshold']
      mask = X[:, feature_idx] < threshold
      X_left, y_left = X[mask], y[mask]
      X_right, y_right = X[~mask], y[~mask]

      majority_label = np.argmax(np.bincount(y))
      node['left'] = self._reduced_error_pruning(node['left'], X_left, y_left, majority_label)
      node['right'] = self._reduced_error_pruning(node['right'], X_right, y_right, majority_label)

      if 'label' in node['left'] and 'label' in node['right']:
          y_pred = self.predict(X)
          
          if y.size == 0:
              node_label = {'label': parent_majority_label}
          else:
              node_label = {'label': np.argmax(np.bincount(y))}
          
          self.tree = node_label
          y_pred_pruned = self.predict(X)

          if np.sum(y_pred != y) >= np.sum(y_pred_pruned != y):
              return node_label

      self.tree = node
      return node

    def _pessimistic_error_pruning(self, node, X, y, n):
         # Realiza poda por erro pessimista
        if 'label' in node:
            node['error'] = np.sum(y != node['label'])
            return node

        feature_idx = node['feature_idx']
        threshold = node['threshold']
        mask = X[:, feature_idx] < threshold
        X_left, y_left = X[mask], y[mask]
        X_right, y_right = X[~mask], y[~mask]

        node['left'] = self._pessimistic_error_pruning(node['left'], X_left, y_left, n)
        node['right'] = self._pessimistic_error_pruning(node['right'], X_right, y_right, n)

        node_error = node['left']['error'] + node['right']['error']
        node['error'] = node_error
        leaf_error = np.sum(y != np.argmax(np.bincount(y)))

        if node_error + np.sqrt(node_error / n) >= leaf_error:
            return {'label': np.argmax(np.bincount(y)), 'error': leaf_error}

        return node

    def prune(self, X, y):
        # Executa a poda da árvore de decisão
        if self.post_pruning == 'reduced_error_pruning':
            self.tree = self._reduced_error_pruning(self.tree, X, y)
        elif self.post_pruning == 'pessimistic_error_pruning':
            self.tree = self._pessimistic_error_pruning(self.tree, X, y)
        else:
            raise ValueError(f"Invalid post_pruning '{self.post_pruning}', use 'reduced_error_pruning' or 'pessimistic_error_pruning'")

# Testes e exemplos para o algoritmo implementado

## Exemplo de uso do código

Este exemplo compara a acurácia de dois classificadores de árvore de decisão de um certo conjunto de dados: um usa a classe personalizada CustomDecisionTreeClassifier implementada anteriormente e o outro usa a implementação do sklearn DecisionTreeClassifier.

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

# Carrega o conjunto de dados
breast_cancer = load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

# Divide o conjunto de dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instancia e ajusta o modelo de árvore de decisão personalizado
dt_custom = CustomDecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1,
                                         pre_pruning='independence', post_pruning='reduced_error_pruning', class_threshold=0.01)
dt_custom.fit(X_train, y_train)

# Faz previsões e calcula a acurácia
y_pred_custom = dt_custom.predict(X_test)
accuracy_custom = accuracy_score(y_test, y_pred_custom)
print(f"CustomDecisionTreeClassifier Accuracy: {accuracy_custom:.2f}")

# Instancia e ajusta o modelo de árvore de decisão do algoritmo do sklearn
dt_sklearn = DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=42)
dt_sklearn.fit(X_train, y_train)

# Faz previsões e calcula a acurácia dado pelo algoritmo do sklearn
y_pred_sklearn = dt_sklearn.predict(X_test)
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Scikit-learn DecisionTreeClassifier Accuracy: {accuracy_sklearn:.2f}")

CustomDecisionTreeClassifier Accuracy: 0.96
Scikit-learn DecisionTreeClassifier Accuracy: 0.95


## Testes "Unittest"

Foram realizados dois testes utilizando o unittest para testar a implementação da árvore de decisão. Temos então o primeiro teste:

Este código contém testes unitários para a classe CustomDecisionTreeClassifier. Ele verifica a correta implementação das funções de cálculo de entropia, índice de Gini, votação majoritária com limite e ajuste/treino e previsão do modelo. O código cria um conjunto de dados de classificação com 100 amostras e 5 features, e verifica se a acurácia do modelo é maior que 0.8 nos dados de teste.

In [6]:
import unittest
from sklearn.datasets import make_classification

class TestCustomDecisionTreeClassifier(unittest.TestCase):
    def setUp(self):
        np.random.seed(42)
        self.X, self.y = make_classification(n_samples=100, n_features=5, n_classes=2)
        self.dt = CustomDecisionTreeClassifier()

    # Testa o cálculo de entropia
    def test_calculate_entropy(self):
        entropy = self.dt._calculate_entropy(np.array([0, 0, 1, 1]))
        self.assertAlmostEqual(entropy, 1.0, places=5)

    # Testa o cálculo do índice de Gini
    def test_calculate_gini(self):
        gini = self.dt._calculate_gini(np.array([0, 0, 1, 1]))
        self.assertAlmostEqual(gini, 0.5, places=5)

    # Testa a votação majoritária com limite
    def test_majority_voting_with_threshold(self):
        majority_class = self.dt._majority_voting_with_threshold(np.array([0, 0, 1, 1]))
        self.assertIsNone(majority_class)

    # Testa o ajuste (fit) e previsão (predict) do modelo
    def test_fit_and_predict(self):
        self.dt.fit(self.X, self.y)
        predictions = self.dt.predict(self.X)
        accuracy = np.sum(predictions == self.y) / len(self.y)
        self.assertGreater(accuracy, 0.8)

# Executa os testes unitários
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


....
----------------------------------------------------------------------
Ran 4 tests in 0.083s

OK


Temos agora o segundo teste:

Este código contém testes unitários adicionais para a classe CustomDecisionTreeClassifier. Ele verifica a correta implementação das funções de cálculo de entropia, índice de Gini, votação majoritária com limite e ajuste/treino e previsão do modelo, tanto com critério de entropia quanto com critério de índice de Gini.

In [8]:
import unittest
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class TestCustomDecisionTreeClassifier(unittest.TestCase):
    def setUp(self):
        np.random.seed(42)
        self.X, self.y = make_classification(n_samples=300, n_features=10, n_classes=3, n_clusters_per_class=1)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.3)
        self.dt = CustomDecisionTreeClassifier()

    # Testa o cálculo de entropia
    def test_calculate_entropy(self):
        entropy = self.dt._calculate_entropy(np.array([0, 0, 1, 1]))
        self.assertAlmostEqual(entropy, 1.0, places=5)

    # Testa o cálculo do índice de Gini
    def test_calculate_gini(self):
        gini = self.dt._calculate_gini(np.array([0, 0, 1, 1]))
        self.assertAlmostEqual(gini, 0.5, places=5)

    # Testa a votação majoritária com limite
    def test_majority_voting_with_threshold(self):
        majority_class = self.dt._majority_voting_with_threshold(np.array([0, 0, 1, 1]))
        self.assertIsNone(majority_class)

    # Testa o ajuste (fit) e previsão (predict) do modelo com critério de entropia
    def test_fit_and_predict(self):
        dt = CustomDecisionTreeClassifier(criterion='entropy', max_depth=None, min_samples_split=2, min_samples_leaf=1)
        dt.fit(self.X_train, self.y_train)
        predictions = dt.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, predictions)
        self.assertGreater(accuracy, 0.8)

    # Testa o processo de poda (pruning) da árvore de decisão
    def test_pruning(self):
        dt = CustomDecisionTreeClassifier(criterion='entropy', max_depth=None, min_samples_split=2, min_samples_leaf=1, post_pruning='reduced_error_pruning')
        dt.fit(self.X_train, self.y_train)
        dt.prune(self.X_test, self.y_test)
        predictions = dt.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, predictions)
        self.assertGreater(accuracy, 0.8)

    # Testa o ajuste (fit) e previsão (predict) do modelo com critério de índice de Gini
    def test_gini_criterion(self):
        dt = CustomDecisionTreeClassifier(criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1)
        dt.fit(self.X_train, self.y_train)
        predictions = dt.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, predictions)
        self.assertGreater(accuracy, 0.8)

# Executa os testes unitários
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 1.485s

OK
